In [ ]:
%%python3 -m venv ../myenv
%%source ../myenv/bin/activate

In [ ]:
%pip install -r ../requirements.txt

In [ ]:
from dotenv import load_dotenv
import pandas as pd
import os
import anthropic

load_dotenv()
ANTHROPIC_API_KEY = os.getenv("ANTHROPIC_API_KEY")

In [ ]:
data_folder = "data"
dev_file_path = os.path.join(data_folder, "merged_instructions.csv")
df = pd.read_csv(dev_file_path)
df['english_instruction'].head()

In [ ]:
result_data = []

for index, row in df.iterrows():
    message = row['english_instruction']

    system_message = f"You are a helpful assistant that generates code snippets based on user instructions." \
                "The method signature should be according to the given in the example part.\n" \
                "Use the following guidlines in generation: \n" \
                " - Provide the code only\n" \
                " - Do not output any other texts or test cases results.\n" \
                " - Include the necessary import statements on the top even if they are usually not needed like - re, typing, itertools, Split etc. \n" \
                " - Do not include any comments in the code\n" \
                " - Do not forget to consider the edge cases\n" \
                " - Avoid recursion if possible\n" \
                " - Lastly, verify the code has no compilations error and contains the necessary imports."
    
    response = anthropic.Anthropic().messages.create(
        model="claude-sonnet-4-0",
        max_tokens=4096,
        system = system_message,
        messages=[
        {
            "role": "user",
            "content": [
                {
                    "type": "text",
                    "text": message
                }
            ]
        }],
        thinking={
            "type": "enabled",
            "budget_tokens": 2048
        }
    )
    output = response.content[1].text
    result_data.append({
        "id": row['id'],
        "response": output
    })

    print("Completed:", row['id'])

result_df = pd.DataFrame(result_data)

In [ ]:
# Create a json from the result_df
result_json = result_df.to_json(orient="records", indent=4)
# save json in the data_folder os.path.join(data_folder, "submission.json")
with open(os.path.join(data_folder, "submission.json"), "w") as json_file:
    json_file.write(result_json)
